In [12]:
import os

# Nom du fichier requis
req_file = '../requirements.txt'

# On vérifie si le fichier existe par rapport à où on est
if os.path.exists(req_file):
    print(f"✅ Fichier trouvé : {req_file}. Installation...")
    %pip install -q -r ../requirements.txt
else:
    print(f"⚠️ Fichier {req_file} introuvable.")
    print("   -> On suppose que l'environnement (GitHub Actions) est déjà prêt.")
    print("   -> Si tu as des erreurs d'import plus bas, c'est que le YAML n'a pas installé les libs.")

✅ Fichier trouvé : ../requirements.txt. Installation...
Note: you may need to restart the kernel to use updated packages.


In [2]:
from food_analyzer import *

In [3]:
saved_dataset = True
df_final = downloader(saved_dataset)
df_final

🌍 Lancement de la collecte
🤖 Utilisation du dataset 
--------------------------------------------------
🚀 DATASET FINAL : 2265 produits.
--------------------------------------------------
Répartition par Catégorie :
Category_Label
Céréales         297
Légumineuses     289
Légumes          289
Pizzas           286
Biscuits         281
Chocolats        278
Jus de Fruits    275
Fromages         270
Name: count, dtype: int64

Répartition par Nutriscore :
Nutriscore
A    455
B    262
C    603
D    486
E    459
Name: count, dtype: int64


,Nom,Marque,Nutriscore,Category_Label,Energie,Sucre,Gras,Saturés,Sel,Fibres,Protéines,Fruits et Légumes
0,Flageolets verts extra fins 4/4,"Marque Repère, Notre Jardin",A,Légumineuses,88.0,0.50,0.600,0.1000,0.580,6.1,5.50,85.334251
1,Cacahuètes grillées sans sel ajouté.,Carrefour,A,Légumineuses,622.0,5.00,51.000,6.1000,0.040,8.6,27.00,95.500000
2,Baked Beans In Tomato Sauce,Essential Waitrose,A,Légumineuses,85.2,3.81,0.476,0.0952,0.505,4.0,7.81,94.762541
3,Tesco No Added Sugar or Salt Swiss Style Muesli,Tesco,A,Céréales,362.0,13.30,4.200,0.8000,0.030,8.5,10.50,19.250000
4,Five Bean Salad,Napolina,A,Légumineuses,124.0,2.10,0.900,0.0000,0.725,6.2,7.10,94.077538
...,...,...,...,...,...,...,...,...,...,...,...,...
2260,Tagger,Bimo,E,Chocolats,535.0,47.00,29.000,18.0000,0.180,1.5,3.30,0.000000
2261,180G C OR NOIR NOISETTE,"Côte D’Or, Mondelez",E,Chocolats,569.0,40.00,39.000,16.0000,0.010,7.8,7.30,24.000000
2262,Abtal,Valencia,E,Jus de Fruits,49.0,11.20,0.000,0.0000,0.000,0.0,0.10,28.000000
2263,Toblerone,Toblerone,E,Chocolats,528.0,60.00,28.000,17.0000,0.120,2.4,5.60,1.600000


Feature Engineering
Sujet : Rétro-ingénierie de l'algorithme du Nutri-Score par approche Machine Learning.

1. Objectif de l'analyse
L'objectif de ce projet est de construire un modèle prédictif capable de déterminer la classe Nutri-Score (A-E) d'un produit à partir de ses données brutes. L'API OpenFoodFacts fournissant plus de 150 variables (métadonnées, emballages, nutriments détaillés), une étape critique de sélection de variables est nécessaire.

Notre approche n'est pas "agnostique" (donner toutes les données au modèle) mais biomimétique : nous cherchons à fournir au modèle les variables qui correspondent aux composantes physico-chimiques réellement utilisées par Santé Publique France, tout en éliminant le bruit statistique.

2. Sélection des Variables Explicatives (Features Retenues)
Nous avons retenu un vecteur de 8 variables quantitatives et 1 variable catégorielle, divisées en trois groupes stratégiques.

A. Les Composantes Négatives (Pénalisantes)

Le Nutri-Score fonctionne par accumulation de "points négatifs" (N). Pour imiter ce comportement, nous avons sélectionné :

nutriments.energy-kcal_100g : La densité énergétique est le premier facteur de pénalité.

Justification : Variable fondamentale. Les variantes en kJ ont été écartées pour éviter la colinéarité parfaite (redondance d'information).

nutriments.sugars_100g (Sucres totaux) :

Justification : Bien que la colonne added-sugars (sucres ajoutés) soit pertinente pour la santé, l'algorithme actuel du Nutri-Score sanctionne les sucres totaux. Utiliser les sucres ajoutés induirait le modèle en erreur sur les produits naturellement sucrés (ex: jus de fruits).

nutriments.saturated-fat_100g (Acides gras saturés) :

Justification Critique : Nous avons délibérément choisi les graisses saturées et non les graisses totales (fat_100g). Un produit peut être très gras mais bien noté (ex: Huile de noix, riche en polyinsaturés). Le modèle doit apprendre à distinguer la "qualité" du gras.

nutriments.sodium_100g (ou équivalent sel) :

Justification : Proxy direct pour le calcul des points liés au sel.

B. Les Composantes Positives (Valorisantes)

Le score est atténué par des "points positifs" (P). Si ces variables sont absentes, le modèle ne peut pas expliquer pourquoi une pizza (B) est mieux notée qu'une autre (D).

nutriments.fiber_100g : Les fibres.

nutriments.proteins_100g : Les protéines.

nutriments.fruits-vegetables-nuts...estimate...100g :

Justification Data Science : C'est souvent une variable latente (calculée à partir de la liste des ingrédients). Sans elle, le modèle échoue systématiquement à prédire correctement les classes A et B des produits transformés mixtes (plats préparés).

C. La Variable de Contextualisation (Le "Switch")

Category_Label (Catégorie du produit) :

Justification Algorithmique : Le calcul du Nutri-Score n'est pas universel. Il existe des grilles de calcul distinctes pour les "Matières grasses", les "Boissons" et les "Fromages".

Impact ML : En incluant la catégorie, nous permettons à un algorithme non-linéaire (comme le Random Forest) de créer des embranchements conditionnels (ex: "Si Fromage, alors tolérer plus de gras").

3. Justification des Variables Rejetées (Réduction de Bruit)
L'exclusion de variables est tout aussi importante pour garantir la robustesse du modèle.

A. Élimination des Micronutriments (Bruit)

Nous avons exclu : vitamin-c, calcium, iron, magnesium, etc.

Raison : Bien que vitaux pour la santé, ces micronutriments n'entrent pas dans la formule du Nutri-Score. Les inclure créerait des corrélations fallacieuses (ex: le modèle pourrait croire que "Vitamine C = Bon score" juste parce que les fruits en contiennent, alors que c'est la teneur en fruits qui compte).

B. Standardisation des Unités (Biais d'échelle)

Nous avons exclu toutes les colonnes _serving (par portion) et _unit.

Raison : Pour que le Machine Learning soit viable, les données doivent être comparables. Comparer 100g de biscuits à une portion de 30g introduirait un biais mathématique majeur. Nous travaillons exclusivement sur la base normalisée Standard 100g.

C. Prévention de la Fuite de Données (Data Leakage)

Nous avons exclu : nutriscore_score (le score numérique, ex: -2, 14).

Raison : C'est la réponse à l'équation. Si nous donnons le score numérique au modèle pour prédire la lettre (A-E), la précision serait de 100%, mais le modèle n'aurait rien appris sur la composition nutritionnelle.

4. Conclusion
Cette sélection de features réduit la dimensionnalité du dataset de 150+ à moins de 10 variables critiques. Elle force le modèle de Machine Learning à "redécouvrir" la pondération officielle du Nutri-Score (les coefficients officiels) plutôt que d'apprendre par cœur des caractéristiques anecdotiques (comme la marque ou l'origine). C'est une approche de Data Science frugale et interprétable.

In [4]:
distribution_nutriments_nutriscore(df_final).show()

In [5]:
impact_matrix_sugar(df_final).show()

In [6]:
corr_matrix(df_final).show()

In [7]:
distribution_nutriscore_par_famille_d_aliments(df_final).show()

In [8]:
nutriscore_par_marques(df_final).show()

In [9]:
sucre_gras(df_final)

👇 Change de catégorie pour mettre à jour instantanément :


interactive(children=(Dropdown(description='Filtre :', options=('Toutes', 'Biscuits', 'Chocolats', 'Céréales',…

In [10]:
model = NutriModel()
accuracy = model.train(df_final)

print(f"🎯 Précision du modèle : {accuracy:.2%}")

print("Test de prédiction")
test_produit = {
    'Energie': 450, 
    'Sucre': 30, 
    'Gras': 20, 
    'Saturés': 10, 
    'Sel': 0.5, 
    'Fibres': 2, 
    'Protéines': 6
}
note = model.predict(test_produit)
print(f"Un produit avec 30g de sucre et 10g de gras saturés aurait la note : {note}")

🎯 Précision du modèle : 99.47%
Test de prédiction
Un produit avec 30g de sucre et 10g de gras saturés aurait la note : E
